In [1]:
#coding:utf-8
import pandas as pd
# from sklearn.preprocessing import OneHotEncoder
import warnings
import xgboost as xgb
import numpy as np
def delUnname0(df):
    df = df.drop('Unnamed: 0',axis=1)
    return df
def xgb_eval_custom_r(y_pred, dtrain):
    y_true = dtrain.get_label()
    up = np.sum(y_pred - y_true)
    down = np.sum(y_pred + y_true)
    whole = np.abs(up/down)
    Loss = whole/len(y_true)
    #here only predict one shop
    return 'Loss', Loss
#preprocessing normailze?No.Do a one hot encoding~~Because the data is so small~~
def TimeFeat(df):
    columns = df.columns
    df['daysinmonth'] = pd.Index(df[columns[1]]).daysinmonth
    df['weekofyear'] = pd.Index(df[columns[1]]).weekofyear
    df['dayofweek'] = pd.Index(df[columns[1]]).dayofweek
    dummies_daysinmonth = pd.get_dummies(df['daysinmonth'],prefix='daysinmonth')
    dummies_weekofyear = pd.get_dummies(df['weekofyear'],prefix='weekofyear')
    dummies_dayofweek = pd.get_dummies(df['dayofweek'],prefix='dayofweek')
    df_dummies = df.join([dummies_daysinmonth,dummies_weekofyear,dummies_dayofweek],how='outer')
    df_dummies = df_dummies.drop(['dayofweek','daysinmonth','weekofyear'],axis=1)
    return df_dummies
def LoadData():
    pd.set_option('display.max_columns', 120)
    Customer_Flow = pd.read_csv("Customer_Flow.csv")
    Customer_Flow = delUnname0(Customer_Flow)
    Customer_Flow['data'] = pd.to_datetime(Customer_Flow['data'])
    return  Customer_Flow
def GenerateAllData(Customer_Flow):
    p_startt = pd.to_datetime("2016-11-01")
    p_endt = pd.to_datetime("2016-11-14")
    p_time = pd.date_range(start=p_startt, end=p_endt, freq='D')
    p_shop_id = range(1, 2001, 1)
    shopid = []
    p_t = []
    for shop_id in p_shop_id:
        for t in p_time:
            shopid.append(shop_id)
            p_t.append(t)
    predict = {'data': p_t, 'shop_id': shopid}
    predict = pd.DataFrame(predict)
    all_data = Customer_Flow.merge(predict, how='outer')
    return all_data
def GenerateTimeF(eachShop):
    eachShop.reset_index(inplace=True)
    eachShop.drop(['index'],inplace=True,axis=1)
    Yesterday_col = {}
    TwoDayAgo_col = {}
    TwoDayAgo_col = pd.DataFrame(TwoDayAgo_col)
    Yesterday_col = pd.DataFrame(Yesterday_col)
    Yesterday_col['yesterday'] = eachShop['Num']
    TwoDayAgo_col['TwoDayAgo_col'] = eachShop['Num']

    eachShop = eachShop.drop(0).drop(1)
    eachShop.reset_index(inplace=True)
    eachShop.drop(['index'],inplace=True,axis=1)

    Yesterday_col = Yesterday_col.drop(0)
    Yesterday_col.reset_index(inplace=True)
    Yesterday_col.drop(['index'],inplace=True,axis=1)

    TwoDayAgo_col.reset_index(inplace=True)
    TwoDayAgo_col.drop(['index'],inplace=True,axis=1)

    newdata = TwoDayAgo_col.join(Yesterday_col)
    newdata = newdata.join(eachShop)
    newdata.drop(['shop_id'],axis=1,inplace=True)
    newdata['difference_two_day'] = newdata['yesterday'] - newdata['TwoDayAgo_col']
    newdata['meanValue14'] =0
    newdata['meanValue7'] =0

    for x in range(14,len(newdata)-15):
        eachline = newdata.ix[x:x]
        fourteen = newdata.ix[x-14:x-1]
        meanValue14 = fourteen['TwoDayAgo_col'].mean()
        meanValue7 = fourteen.ix[len(fourteen)-7+x-14:len(fourteen)-1+x-14]
        meanValue7 = meanValue7['TwoDayAgo_col'].mean()
        eachline['meanValue14'] = meanValue14
        eachline['meanValue7'] = meanValue7
        newdata.update(eachline)
    newdata.drop(len(newdata)-1,inplace=True)
    newdata.drop(len(newdata)-1,inplace=True)
    for x in range(0,14):
        newdata.drop(x,inplace=True)

    newdata.reset_index(inplace=True)
    newdata.drop(['index'],axis=1,inplace=True)

    basePredict = newdata.ix[len(newdata)-14-13:len(newdata)-1]
    for x in range(0,14):
        newdata.drop(len(newdata)-1,inplace=True)
    return basePredict,newdata
def SplitData(PredictData,newdata):
    PredictData = PredictData.reset_index()
    PredictData.drop(['index'],axis=1,inplace=True)
    train_data = newdata.ix[0:int(0.9*len(newdata))]
    test_data = newdata.ix[int(0.9*len(newdata)):len(newdata)-1]
    return PredictData,train_data,test_data
def FittingModel(train_data,test_data,ToDrop,shop_id):
    train_xgb = xgb.DMatrix(data=train_data.drop(ToDrop,axis=1)
                            ,label=train_data['Num'])
    valid_xgb = xgb.DMatrix(data=test_data.drop(ToDrop,axis=1)
                            ,label=test_data['Num'])

    params = {
      'objective': 'reg:linear'
      ,'eta': 0.1
      ,'max_depth': 6
      , 'subsample': 0.4
      , 'colsample_bytree': 0.9
      ,'min_child_weight': 12
      ,'gamma': 0.07
      , 'seed': 10
    ,'reg_alpha': 0.06
    }
    evallist = [(train_xgb, 'train'), (valid_xgb, 'valid')]
    model = xgb.train(params.items()
                      , dtrain=train_xgb
                      , num_boost_round=10000
                      , evals=evallist
                      , early_stopping_rounds=20
                      , maximize=False
                      , verbose_eval=10
                      , feval=xgb_eval_custom_r
                  )
    print "now it is:",shop_id
    print ('get info from model')
    print (model.best_score, model.best_iteration,model.best_ntree_limit)
    return model

In [3]:
p_result = []
Customer_Flow = LoadData()
all_data = GenerateAllData(Customer_Flow)

# for shop_id, eachShop in all_data.groupby(['shop_id']):
eachShop = all_data[all_data['shop_id'] ==1]
shop_id =1;
warnings.filterwarnings("ignore")
eachShop = TimeFeat(eachShop)
PredictData, newdata = GenerateTimeF(eachShop)

In [4]:
newdata

,TwoDayAgo_col,yesterday,Num,data,daysinmonth_29,daysinmonth_30,daysinmonth_31,weekofyear_1,weekofyear_2,weekofyear_3,weekofyear_4,weekofyear_5,weekofyear_6,weekofyear_7,weekofyear_8,weekofyear_9,weekofyear_10,weekofyear_11,weekofyear_12,weekofyear_13,weekofyear_14,weekofyear_15,weekofyear_16,weekofyear_17,weekofyear_18,weekofyear_19,weekofyear_20,weekofyear_21,weekofyear_22,weekofyear_23,weekofyear_24,weekofyear_25,weekofyear_26,weekofyear_27,weekofyear_28,weekofyear_29,weekofyear_30,weekofyear_31,weekofyear_32,weekofyear_33,weekofyear_34,weekofyear_35,weekofyear_36,weekofyear_37,weekofyear_38,weekofyear_39,weekofyear_40,weekofyear_41,weekofyear_42,weekofyear_43,weekofyear_44,weekofyear_45,weekofyear_46,weekofyear_47,weekofyear_48,weekofyear_49,weekofyear_50,weekofyear_51,weekofyear_52,weekofyear_53,dayofweek_0,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6,difference_two_day,meanValue14,meanValue7
0,291.0,286.0,264.0,2015-10-26,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-5.0,255.357143,267.571429
1,286.0,264.0,295.0,2015-10-27,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-22.0,262.714286,267.285714
2,264.0,295.0,310.0,2015-10-28,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,31.0,263.714286,266.571429
3,295.0,310.0,212.0,2015-10-29,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,15.0,264.214286,267.714286
4,310.0,212.0,331.0,2015-10-30,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-98.0,268.214286,272.428571
5,212.0,331.0,347.0,2015-10-31,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,119.0,270.571429,278.000000
6,331.0,347.0,381.0,2015-11-01,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,16.0,269.500000,271.142857
7,347.0,381.0,332.0,2015-11-02,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,34.0,275.857143,284.142857
8,381.0,332.0,404.0,2015-11-03,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-49.0,279.714286,292.142857
9,332.0,404.0,392.0,2015-11-04,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0

In [5]:
PredictData

,TwoDayAgo_col,yesterday,Num,data,daysinmonth_29,daysinmonth_30,daysinmonth_31,weekofyear_1,weekofyear_2,weekofyear_3,weekofyear_4,weekofyear_5,weekofyear_6,weekofyear_7,weekofyear_8,weekofyear_9,weekofyear_10,weekofyear_11,weekofyear_12,weekofyear_13,weekofyear_14,weekofyear_15,weekofyear_16,weekofyear_17,weekofyear_18,weekofyear_19,weekofyear_20,weekofyear_21,weekofyear_22,weekofyear_23,weekofyear_24,weekofyear_25,weekofyear_26,weekofyear_27,weekofyear_28,weekofyear_29,weekofyear_30,weekofyear_31,weekofyear_32,weekofyear_33,weekofyear_34,weekofyear_35,weekofyear_36,weekofyear_37,weekofyear_38,weekofyear_39,weekofyear_40,weekofyear_41,weekofyear_42,weekofyear_43,weekofyear_44,weekofyear_45,weekofyear_46,weekofyear_47,weekofyear_48,weekofyear_49,weekofyear_50,weekofyear_51,weekofyear_52,weekofyear_53,dayofweek_0,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6,difference_two_day,meanValue14,meanValue7
358,207.0,244.0,177.0,2016-10-19,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,37.0,252.928571,247.428571
359,244.0,177.0,207.0,2016-10-20,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-67.0,247.714286,242.857143
360,177.0,207.0,230.0,2016-10-21,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,30.0,245.357143,241.428571
361,207.0,230.0,202.0,2016-10-22,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,23.0,239.571429,232.571429
362,230.0,202.0,243.0,2016-10-23,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-28.0,237.785714,229.571429
363,202.0,243.0,216.0,2016-10-24,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,41.0,235.714286,227.571429
364,243.0,216.0,255.0,2016-10-25,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-27.0,232.500000,218.714286
365,216.0,255.0,172.0,2016-10-26,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,39.0,231.571429,215.714286
366,255.0,172.0,226.0,2016-10-27,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-83.0,229.928571,217.000000
367,172.0,226.0,257.0,2016-10-28,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [ ]:
PredictData,train_data, test_data = SplitData(PredictData,newdata)
    ToDrop = ['Num', 'data']
    print "fitting data on model....."
    model =  FittingModel(train_data,test_data,ToDrop,shop_id)
    result = []
    dataLen = len(PredictData)

In [ ]:
print "predicting...."
    for x in range(13, dataLen):
        first = PredictData.ix[x:x]
        if x == 14:
            first['yesterday'] = firstNum
            Next_TwoDayAgo_col = firstNum
            fourteen = PredictData.ix[x-14:x-1]
            meanValue14 = fourteen['TwoDayAgo_col'].mean()
            meanValue7 = fourteen.ix[len(fourteen)-7+x-14:len(fourteen)-1+x-14]
            meanValue7 = meanValue7['TwoDayAgo_col'].mean()
            first['meanValue14'] = meanValue14
            first['meanValue7'] = meanValue7
        if x >= 15:
            first['yesterday'] = firstNum
            first['TwoDayAgo_col'] = Next_TwoDayAgo_col
            Next_TwoDayAgo_col = firstNum
            fourteen = PredictData.ix[x-14:x-1]
            meanValue14 = fourteen['TwoDayAgo_col'].mean()
            meanValue7 = fourteen.ix[len(fourteen)-7+x-14:len(fourteen)-1+x-14]
            meanValue7 = meanValue7['TwoDayAgo_col'].mean()
            first['meanValue14'] = meanValue14
            first['meanValue7'] = meanValue7        

        predict_xgb = xgb.DMatrix(data=first.drop(ToDrop, axis=1))
        firstNum = model.predict(predict_xgb, ntree_limit=model.best_ntree_limit)
        result.append(firstNum)
    result = pd.DataFrame(result)
    result = result.rename(columns={0:'p'})
    each_line = []
    each_line.append(shop_id)
    for Num in result.p:
        each_line.append(Num)
    p_result.append(each_line)
    print " the task  of ",shop_id,"is finished!!"

In [ ]:
p_result = abs(pd.DataFrame(p_result).astype(int))
p_result.to_csv('LR.csv',header=False,index=False,encoding='utf-8')